In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm
import math
from PCCNNv4 import PCCNNModel
from CNN import CNNModel
from CustomDataset import LoadFromFolder
import numpy as np

c:\Users\Joe\anaconda3\envs\ml-env\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [3]:
# BATCH_SIZE = 1024
# train_dataset = datasets.MNIST(root='../../datasets/', train=True, transform=transforms.ToTensor(), download=True)
# train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=BATCH_SIZE)
# test_dataset = datasets.MNIST(root='../../datasets/', train=False, transform=transforms.ToTensor(), download=True)
# test_loader = DataLoader(dataset=test_dataset, shuffle=True, batch_size=BATCH_SIZE)
# INPUT_CHANNELS = 1
# NUM_CLASSES = 10

In [4]:
# BATCH_SIZE = 500
# train_val_dataset = datasets.CIFAR10(root='../../datasets/', train=True, transform=transforms.ToTensor(), download=True)
# train_dataset, val_dataset = torch.utils.data.random_split(train_val_dataset, [45000, 5000])
# train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=BATCH_SIZE)
# val_loader = DataLoader(dataset=val_dataset, shuffle=True, batch_size=BATCH_SIZE)
# test_dataset = datasets.CIFAR10(root='../../datasets/', train=False, transform=transforms.ToTensor(), download=True)
# test_loader = DataLoader(dataset=test_dataset, shuffle=True, batch_size=BATCH_SIZE)
# INPUT_CHANNELS = 3
# NUM_CLASSES = 10

In [5]:
def topk_accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)
    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].reshape(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size).item())
    return res

In [6]:
def train(model, optimiser, criterion, model_name, num_epochs, flatten=False, cross_val=False, log_dir="logs"):
    writer = SummaryWriter(f"{log_dir}/{model_name}_Epochs{num_epochs}_BS{BATCH_SIZE}_LR{LEARNING_RATE}_WD{WEIGHT_DECAY}_")
    train_loss = []
    train_acc = []
    train_e_mag = []
    val_loss = []
    val_acc = []
    val_e_mag = []
    
    best_val_acc = 0.0
    best_val_loss = 99999999.9

    val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE)
    for epoch in range(num_epochs):
        train_dataset.apply_transform()
        
        train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=BATCH_SIZE)
        loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)    

        num_correct = 0
        total_train_loss = 0
        total_train_e_mag = 0

        for batch_idx, (images, y) in loop:
            x = images.to(device)
            if flatten:
                x = torch.flatten(x, start_dim=1)
            target = y.to(device)
            out = model(x)

            loss = criterion(out[0], target)
            optimiser.zero_grad()
            loss.backward()
            optimiser.step()

            with torch.no_grad():
                total_train_loss += loss.item()
                total_train_e_mag += out[1]

                prediction = torch.argmax(out[0], dim=1)
                batch_correct = (prediction == y.to(device)).sum().item()
                num_correct += batch_correct
                n = x.shape[0]

                if epoch > 0:
                    loop.set_description(f"Epoch [{epoch}/{num_epochs}]")
                    loop.set_postfix(
                        train_loss = train_loss[-1], 
                        train_acc = train_acc[-1], 
                        val_loss = val_loss[-1], 
                        val_acc = val_acc[-1][0],
                    )


        with torch.no_grad():
            model.eval()
            train_loss.append(total_train_loss / len(train_loader))
            train_acc.append(num_correct * 100.0 / len(train_dataset))
            train_e_mag.append(total_train_e_mag / len(train_loader))

            val_correct = 0
            val_n = 0
            total_val_loss = 0
            total_val_acc = torch.zeros(3)
            total_val_e_mag = 0

            for batch_idx, (images, y) in enumerate(val_loader):
                x = images.to(device)
                if flatten:
                    x = torch.flatten(x, start_dim=1)
                target = y.to(device)
                out = model(x)
                total_val_loss += criterion(out[0], target).item()
                total_val_acc += torch.tensor(topk_accuracy(out[0], target, (1,3,5)))
                total_val_e_mag += out[1]
                
            val_loss.append(total_val_loss / len(val_loader))
            val_acc.append((total_val_acc / len(val_loader)).tolist())    
            val_e_mag.append(total_val_e_mag / len(val_loader))
                
#             if best_val_acc < val_acc[-1][0]:
#                 best_val_acc = val_acc[-1][0]
#                 torch.save(model.state_dict(), f'models/{model_name}.pth')
                
            if best_val_loss > val_loss[-1]:
                best_val_loss = val_loss[-1]
                torch.save(model.state_dict(), f'models/{model_name}.pth')

            step = epoch * len(train_dataset)
            writer.add_scalar("Training Loss", train_loss[-1], step)
            writer.add_scalar("Training Accuracy", train_acc[-1], step)
            writer.add_scalar("Training E_mag", train_e_mag[-1], step)
            writer.add_scalar("Validation Loss", val_loss[-1], step)
            writer.add_scalar("Validation Accuracy Top1", val_acc[-1][0], step)
            writer.add_scalar("Validation E_mag", val_e_mag[-1], step)
            writer.add_scalar("Validation Accuracy Top3", val_acc[-1][1], step)
            writer.add_scalar("Validation Accuracy Top5", val_acc[-1][2], step)
            
            
            model.train()
    return np.array(train_loss), np.array(train_acc), np.array(val_loss), np.array(val_acc)

In [7]:
class RandomGaussianNoise(object):
    def __init__(self, mean=0.0, std=0.001):
        self.mean = mean
        self.std = std
    
    def __call__(self, img):
        noise = (torch.randn(img.shape) * self.std + self.mean)
        if img.is_cuda:
            noise = noise.to("cuda")
        return img + noise

In [8]:
transform_arr = [
    transforms.Compose([ 
#         transforms.ToTensor(), 
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomAffine(degrees=90, translate=(0.15,0.15), scale=(0.8,1.2))]),
    transforms.Compose([ 
#         transforms.ToTensor(), 
        transforms.RandomHorizontalFlip(p=0.5), 
        transforms.ColorJitter(hue=0.015, saturation=0.015, brightness=0.015)]),
    transforms.Compose([ 
#         transforms.ToTensor(), 
        transforms.RandomHorizontalFlip(p=0.5), 
        transforms.GaussianBlur((5,5), sigma=(0.01, 0.75))]),
    transforms.Compose([ 
#         transforms.ToTensor(), 
        transforms.RandomHorizontalFlip(p=0.5), 
        transforms.RandomErasing(scale=(0.02, 0.3))]),
    transforms.Compose([ 
#         transforms.ToTensor(), 
        transforms.RandomHorizontalFlip(p=0.5),
        RandomGaussianNoise()])
]

In [10]:
BATCH_SIZE = 150
INPUT_SHAPE = (3,64,64)
NUM_CLASSES = 30
VAL_RATIO = 0.2
LEARNING_RATE = 3e-4
WEIGHT_DECAY=3e-2
NUM_EPOCHS=100
FEATURES=16

dataset = LoadFromFolder('..\\..\\..\\datasets\\TinyImageNet30\\train_set\\train_set', 
                         (0,INPUT_SHAPE[0], INPUT_SHAPE[1], INPUT_SHAPE[2]), 
                         transform=transform_arr[0],
                         shuffle=False,
                        )

In [11]:
train_loss_arr, train_acc_arr, val_loss_arr, val_acc_arr = [], [], [], [] 
criterion = nn.CrossEntropyLoss()
for i in range(1):
    train_dataset, val_dataset = dataset.cross_val_split_by_class(VAL_RATIO, i, device="cpu")
    # train_dataset.transform = transform_arr[i]
    print(len(train_dataset))
    print(len(val_dataset))
    train_loader = DataLoader(dataset=train_dataset, shuffle=True, batch_size=BATCH_SIZE)
    val_loader = DataLoader(dataset=val_dataset, batch_size=BATCH_SIZE)
    
    model_name = f"PCCNNv4_{str(FEATURES)}_{i}"
    model = PCCNNModel(FEATURES, INPUT_SHAPE, NUM_CLASSES, device="cpu")
    model.to(device)
    print(type(model))
    optimiser = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
    train_loss, train_acc, val_loss, val_acc = train(
        model, 
        optimiser, 
        criterion, 
        model_name, 
        num_epochs=NUM_EPOCHS, 
        cross_val=True,
        log_dir="logs_pccnnv4"
    )
    
    train_loss_arr.append(train_loss)
    train_acc_arr.append(train_acc)
    val_loss_arr.append(val_loss)
    val_acc_arr.append(val_acc)

10800
2700
<class 'PCCNNv4.PCCNNModel'>


KeyboardInterrupt: 

In [ ]:
def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.ConvTranspose2d, nn.BatchNorm2d)):
            nn.init.normal_(m.weight.data, 0.0, 0.02)

def test():
    
    N, inp_channels, H, W, NUM_CLASSES = 8, 3, 32, 32, 10
    z_dim = 100
    x = torch.randn((N, inp_channels, H, W)).to(device)
    cnn = CNNModel().to(device)
    initialize_weights(cnn)
    assert cnn(x)[0].shape == (N,10)
    
    pccnn = PCModel().to(device)
    initialize_weights(pccnn)
    assert pccnn(x)[0].shape == (N,10)
    
test()

In [5]:
def evaluate(model, with_targets=False):
    model.eval()
    n = 0
    correct = 0
    for batch_idx, (data, y) in enumerate(test_loader):
#         x = data[:,:,:27,:27].to(device)
        x = data.to(device)
        batch_size = data.shape[0]
        n = n + batch_size
        target = None
        if with_targets:
            target = F.one_hot(y).unsqueeze(2).unsqueeze(3).to(device)
        out = model(x, target)
        output = torch.argmax(out[0], dim=1)
        y = y.to(device)
        batch_correct = output == y
        correct = correct + batch_correct.sum()

    print(f'Test Accuracy: {correct/n}')

In [ ]:
model.load_state_dict(torch.load('PC-RB_Conv2D-V2.pth'))
model.eval()

In [6]:
model_name = 'PCCNN_ignore-emag'
writer = SummaryWriter(f"logs/{model_name}")

In [8]:
LEARNING_RATE = 3e-4
NUM_EPOCHS = 100
WEIGHT_DECAY = 0.02

# optimiser = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
optimiser = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)
criterion = nn.CrossEntropyLoss().to(device)
n = 0
correct = 0
val_loss = 0
model.train()
for epoch in range(NUM_EPOCHS, NUM_EPOCHS*2):
    
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=False)
    
    train_loss = 0
    train_batches = 0
    train_emag = 0
    
    for batch_idx, (data, y) in loop:
#         x = data[:,:,:27,:27].to(device)
        x = data.to(device)
        target = F.one_hot(y).double().to(device)
        out = model(x)
        loss = criterion(out[0], target)
#         if len(out) > 1:
#             loss += out[1]
        
        loss.backward(retain_graph=True)
        optimiser.step()
        optimiser.zero_grad()
        
        with torch.no_grad():
            output = torch.argmax(out[0], dim=1)
            arg_correct = output == y.to(device)
            correct = correct + arg_correct.sum()
            n = n + x.shape[0]
            
            train_loss += loss.item()
            train_batches += 1
            if len(out) > 1:
                train_emag += out[1].item()
            
            # Update progress bar
            loop.set_description(f"Epoch [{epoch}/{NUM_EPOCHS}]")
            loop.set_postfix(loss = loss.item(), val_loss = val_loss, train_acc=(correct/n).item())
    
    # Calculate validation_acc
    with torch.no_grad():
        val_emag = 0
        val_n = 0
        val_correct = 0
        val_batches = 0
        for batch_idx, (data, y) in enumerate(val_loader):
            x = data.to(device)
            batch_size = data.shape[0]
            val_n += batch_size
            out = model(x)
            output = torch.argmax(out[0], dim=1)
            y = y.to(device)
            batch_correct = output == y
            val_correct += batch_correct.sum()
            if len(out) > 1:
                val_emag += out[1].item()
            val_batches += 1
            
        train_loss /= train_batches
        val_acc = val_correct/val_n
        
        # Update logs
        writer.add_scalar('training loss', train_loss, epoch*train_batches*BATCH_SIZE)
        writer.add_scalar('validation accuracy', val_acc, epoch*train_batches*BATCH_SIZE)
        
        if len(out) > 1:
            train_emag /= train_batches
            val_emag /= val_batches
            writer.add_scalar('training emag', train_emag, epoch*train_batches*BATCH_SIZE)
            writer.add_scalar('validation emag', val_emag, epoch*train_batches*BATCH_SIZE)
        
    

In [ ]:
evaluate(model)

In [ ]:
torch.save(model.state_dict(), 'PC-RB_Conv2D-V2.pth')

In [ ]:
LEARNING_RATE = 3e-7
NUM_EPOCHS = 5
optimiser = optim.SGD(model.parameters(), lr=LEARNING_RATE)
mean_loss = 0

model.eval()
for epoch in range(NUM_EPOCHS):
    for batch_idx, (data, y) in enumerate(train_loader):
        x = data[:,:,:27,:27].to(device)
        out = model(x)
        break
    break

In [ ]:
x = torch.tensor([1, 2, 3, 4, 1, 5,1, 9, 5, 8, 0, 2, 6, 5, 8, 2])
x = F.argmax(x)
x